In [1]:
%matplotlib inline
import numpy as np

from transform import combine_tsv_files, load_process_and_store_spreadsheets

In [2]:
def harmonic_mean(t0, t1):
    return t0 * t1 / (t0 + t1)

load_process_and_store_spreadsheets(
    transformation=harmonic_mean,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

/home/hylke/.local/lib/python3.8/site-packages/pandas/core/indexes/multi.py:3366: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  uniq_tuples = lib.fast_unique_multiple([self._values, other._values], sort=sort)


> /home/hylke/workspace/avenio/transform.py(309)load_process_and_store_spreadsheets()
    308     # Make a different document for each of the column pairs.
--> 309     for column in columns:
    310         column_pair = [f"T0: {column}", f"T1: {column}"]

ipdb> n
> /home/hylke/workspace/avenio/transform.py(310)load_process_and_store_spreadsheets()
    309     for column in columns:
--> 310         column_pair = [f"T0: {column}", f"T1: {column}"]
    311         # Filter out the column pairs, and remove empty fields.

ipdb> 
> /home/hylke/workspace/avenio/transform.py(312)load_process_and_store_spreadsheets()
    311         # Filter out the column pairs, and remove empty fields.
--> 312         mutations = spread_sheet[column_pair].dropna(how="all").fillna(0)
    313         # Repair dirty cells (with per cent signs etc.).

ipdb> 
> /home/hylke/workspace/avenio/transform.py(314)load_process_and_store_spreadsheets()
    313         # Repair dirty cells (with per cent signs etc.).
--> 3

BdbQuit: 

In [23]:
def difference(t0, t1):
    return t1 - t0

load_process_and_store_spreadsheets(
    transformation=difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

In [24]:
def relative_difference(t0, t1):
    if t0 != 0.0:
        return (t1-t0)/t0
    return (t1 - t0)/t1

load_process_and_store_spreadsheets(
    transformation=relative_difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [2]:
def up_or_down(t0, t1):
    return np.sign(t1 - t0)


load_process_and_store_spreadsheets(
    transformation=up_or_down,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

/home/hylke/.local/lib/python3.8/site-packages/pandas/core/indexes/multi.py:3366: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  uniq_tuples = lib.fast_unique_multiple([self._values, other._values], sort=sort)


In [26]:
def margin_sign(t0, t1, margin=0.15):
    if t0 == 0.0:  # Complete clearance at baseline.
        return 1
    if t1 == 0.0:  # Complete clearance at follow up.
        return -1
    dt = (t1-t0)/t0
    if dt > margin:
        return 1
    elif dt < -margin:
        return -1
    return 0


load_process_and_store_spreadsheets(
    transformation=margin_sign,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Data without train/test split, with CNV scores merged in.

In [ ]:
def to_excel(snv_name, cnv_name):
    excel_filename = snv_name.replace('.tsv', '_CNV.xlsx')
    X, y = combine_tsv_files(snv_name, cnv_name)
    X.merge(y, left_index=True, right_index=True).to_excel(excel_filename)
    print(f'Wrote {excel_filename}.')

In [ ]:
for coarseness in ('gene', 'pathway'):
    for transformation in ('harmonic_mean', 'difference', 'relative_difference', 'up_or_down', 'margin_sign'):
        to_excel(
            f"output/all__{coarseness}__{transformation}__Allele Fraction.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )
        to_excel(
            f"output/all__{coarseness}__{transformation}__No. Mutant Molecules per mL.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )